# Setup

### TODO:
-  ADD the training sequences (see how to get it on : https://github.com/pinellolab/DNA-Diffusion/blob/main/src/dnadiffusion/data/K562_hESCT0_HepG2_GM12878_12k_sequences_per_group.txt). Capture 200 nucleotide regions from the center of these coordinates (check bedtools)
-  Lets try to have table with all the endogenous sequences, but for NOW just 1k regions from the training set


In [ ]:
import tensorflow_hub as hub
import joblib
import gzip
import kipoiseq
from kipoiseq import Interval
import pyfaidx
import pyBigWig
import igv_notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import os 
import pickle as pkl
import wget 
from tqdm import tqdm_notebook as tqdm
import tensorflow as tf
from enformer_dna_diff.enformer import Enformer, FastaStringExtractor
from enformer_dna_diff.enformerops import EnformerOps, SEQ_EXTRACT

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# assert tf.config.list_physical_devices('GPU'), 'Start the colab kernel with GPU: Runtime -> Change runtime type -> GPU'

# tf.config.list_physical_devices('GPU')

In [ ]:
!ls

5_31_2023_enformer_CAGE_DNAse_bias_experiment.ipynb
CAGE:GM12878_enformer.bw
CAGE:HepG2_enformer.bw
CAGE:K562_enformer.bw
DNASE:GM12878_enformer.bw
DNASE:H1esc_enformer.bw
DNASE:HepG2_enformer.bw
DNASE:K562_enformer.bw
DNASE_RANDOM_SEQS.TXT
DNA_DIFFUSION_ALL_SEQS.txt
Enformer_Experiments
GENERATED_SEQS.TXT
MASTER_DNA_DIFFUSION_ALL_SEQS.txt
PROMOTERS_SEQS.TXT
TRAINING_TEST_VALIDATION_SEQS.TXT
Untitled.ipynb
bin
boot
data
demo_out.ipynb
demo_out_tq.ipynb
dev
etc
hg38.chrom.sizes
hg38_liftover+new_CAGE_peaks_phase1and2.bed.gz
home
lib
lib64
media
mnt
opt
proc
root
run
sbin
srv
sys
tmp
tqdm.ipynb
usr
var


In [ ]:

model_path = 'https://tfhub.dev/deepmind/enformer/1'
fasta_file = 'data/genome.fa'
targets_txt = 'https://raw.githubusercontent.com/calico/basenji/master/manuscripts/cross2020/targets_human.txt'
df_targets = pd.read_csv(targets_txt, sep='\t')
df_targets[df_targets.apply(lambda x : '' in x['description'] and  'CAGE' in x['description'] , 1)]
df_sizes = pd.read_table('data/hg38.chrom.sizes', header=None).head(22)
SEQUENCE_LENGTH = 393216
model = Enformer(model_path)
fasta_extractor = FastaStringExtractor(fasta_file)

In [ ]:
SEQS = SEQ_EXTRACT('data/MASTER_DNA_DIFFUSION_ALL_SEQS.txt')
SEQS

chrom   start     end      ID  SEQUENCE
TAG                   CELL_TYPE                                          
GENERATED             GM12878    100000  100000  100000  100000    100000
                      HEPG2      100000  100000  100000  100000    100000
                      HESCT0     100000  100000  100000  100000    100000
                      K562       100000  100000  100000  100000    100000
PROMOTERS             NO          20013   20013   20013   20013     20013
RANDOM_GENOME_REGIONS NO           5000    5000    5000    5000      5000
test                  GM12878      1044    1044    1044    1044      1044
                      HepG2         958     958     958     958       958
                      K562         1321    1321    1321    1321      1321
                      hESCT0       1242    1242    1242    1242      1242
training              GM12878      9903    9903    9903    9903      9903
                      HepG2        9833    9833    9833    9833      9833
                      K562         9769    9769    9769    9769      9769
                      hESCT0       9749    9749    9749    9749      9749
validation            GM12878      1021    1021    1021    1021      1021
                      HepG2        1177    1177    1177    1177      1177
                      K562          878     878     878     878       878
                      hESCT0        977     977     977     977       977

Data structure

In [ ]:
# -Add the random for our full dataset.
# -Dont remove hesct0 (use h1esc) use NA for cage
# -Generate 10k random sequences (remove regions with NN)
# -Add the CAGE tss(1kb+/-) prediction for the other 3 celltypes on the 440k.
# -Dont show tss in the final sequences.

In [ ]:
# eops.add_track({
#     'name': 'DNAse:hESC_enformer',
#     'file': None,
#     'color': 'BLUE',
#     'type': 'enformer',
#     'id': 123,
#     'log': False
# })

def load_tracks_eops(eops):
    
    eops.add_track({
        'name': 'DNASE:GM12878_enformer',
        'file': None,
        'color': 'BLUE',
        'type': 'enformer',
        'id': 12,
        'log': False
    })

    eops.add_track({
        'name': 'DNASE:K562_enformer',
        'file': None,
        'color': 'BLUE',
        'type': 'enformer',
        'id': 121,
        'log': False
    })

    eops.add_track({
        'name': 'DNASE:HepG2_enformer',
        'file': None,
        'color': 'BLUE',
        'type': 'enformer',
        'id': 27,
        'log': False
    })
    
    eops.add_track({
        'name': 'DNASE:H1esc_enformer',
        'file': None,
        'color': 'BLUE',
        'type': 'enformer',
        'id': 19,
        'log': False
    })

    eops.add_track({
        'name': 'CAGE:K562_enformer',
        'file': None,
        'color': 'RED',
        'type': 'enformer',
        'id': 5111,
        'log': True
    })

    eops.add_track({
        'name': 'CAGE:GM12878_enformer',
        'file': None,
        'color': 'RED',
        'type': 'enformer',
        'id': 5110,
        'log': True
    })

    eops.add_track({
        'name': 'CAGE:HepG2_enformer',
        'file': None,
        'color': 'RED',
        'type': 'enformer',
        'id': 5109,
        'log': True
    })
    
    # eops.add_track({
    #     'name': 'CAGE:hESC_enformer',
    #     'file': None,
    #     'color': 'RED',
    #     'type': 'enformer',
    #     'id': 500,
    #     'log': True
    # })

    # eops.add_track({
    #     'name': 'K562 DNASE REAL',
    #     'file': 'ENCFF413AHU.bigWig',
    #     'color': 'skyblue',
    #     'type': 'real'
    # })
    # eops.add_track({
    #     'name': 'GM12878 DNASE REAL',
    #     'file': 'ENCFF093VXI.bigWig',
    #     'color': 'skyblue',
    #     'type': 'real'
    # })

    # eops.remove_track(['CAGE:hESC_enformer', 'DNAse:hESC_enformer'])

# Enformer output analysis of training; random; and generated sequences

In [ ]:
# all_data

In [ ]:
# 6 hours  random
# 20 hours promoters 
# 50 hours train/test/validation
# 350 hours  GENERATED

In [ ]:
DEMO=True
all_data = SEQS.data.copy()
MODIFY_PREFIX= '1_'
#run lucas_test
#papermill  5_31_2023_enformer_CAGE_DNAse_bias_experiment.ipynb  demo.ipynb

## Random sequences

In [ ]:
print('Remove all head 3')
eops = EnformerOps()
eops.load_data(SEQS.extract_seq('GENERATED', 'GM12878')['SEQUENCE'].values.tolist())
load_tracks_eops(eops) #loading the tracks

ran_seqs = all_data[all_data['TAG'] == 'RANDOM_GENOME_REGIONS'].copy()
print (ran_seqs['TAG'].unique(), ran_seqs.shape)

subset_ran_seqs = ran_seqs[['chrom', 'start', 'end', 'ID']]
#ran_seq_list = subset_ran_seqs.head(3).values.tolist() # removing demo

ran_seq_list = subset_ran_seqs.values.tolist() # removing demo
if DEMO:
    ran_seq_list=ran_seq_list[:3]

captured_values = []
for s in ran_seq_list: 
    #print(s)
    try:
        s_in = [s[0], int(s[1]), int(s[2])]
        id_seq = s[3] 
        list_bw = eops.generate_plot_number(0, interval_list=s_in, wildtype=True, 
                                          show_track=False, modify_prefix=MODIFY_PREFIX) 
    except RuntimeError:
      # Infrequent "the entries are out of order" error for some random seqs 
        continue
    try:
        out_in = eops.extract_from_position(s_in, as_dataframe=True)
        #print (out_in.shape)
        out_in = out_in.mean()
        out_in['SEQ_ID'] = id_seq
        out_in['TARGET_NAME'] = 'ITSELF'
        columns_names = out_in.copy()
        captured_values.append( out_in  )
    except ValueError:
      # Infrequent "All arrays must be of the same length" error
        continue

df_out = pd.DataFrame([x.values.tolist() for x in captured_values], columns=out_in.index)

df_out.to_csv('DNASE_RANDOM_SEQS.TXT', sep='\t', index=None)

Remove all head 3


['RANDOM_GENOME_REGIONS'] (5000, 7)


## Promoters

In [ ]:
# ['RANDOM_GENOME_REGIONS']        (5000)    (MEASURING: 200bp)       #DNASE (K562,HEPG2,GM, H1ESC) # CAGE (K562,HEPG2,GM)
# ['PROMOTERS']                    (20013)  (MEASURING: 2000bp)       #DNASE (K562,HEPG2,GM, H1ESC) # CAGE (K562,HEPG2,GM)
# ['training' 'test' 'validation'] (47872)   (MEASURING: 200bp)       #DNASE (K562,HEPG2,GM, H1ESC) # CAGE (K562,HEPG2,GM)

# #2X (ENHANCER AND GATA1)  #DNASE (K562,HEPG2,GM, H1ESC) # CAGE (K562,HEPG2,GM)
# ['GENERATED']                    (400000)   (MEASURING: ENH(200bp) GATA1_GB( 7925bp) ) 
                            


In [ ]:
print('Remove all head 3')
eops = EnformerOps()
eops.load_data(SEQS.extract_seq('GENERATED', 'GM12878')['SEQUENCE'].values.tolist())
load_tracks_eops(eops) #loading the tracks

ran_seqs = all_data[all_data['TAG'] == 'PROMOTERS'].copy()
print (ran_seqs['TAG'].unique(), ran_seqs.shape)

subset_ran_seqs = ran_seqs[['chrom', 'start', 'end', 'ID']]
#ran_seq_list = subset_ran_seqs.head(3).values.tolist()
ran_seq_list = subset_ran_seqs.values.tolist()
if DEMO:
    ran_seq_list=ran_seq_list[:3]

captured_values = []
for s in tqdm(ran_seq_list): 
    #print(s)
    try:
        s_in = [s[0], int(s[1]), int(s[2])]
        id_seq = s[3] 
        list_bw = eops.generate_plot_number(0, interval_list=s_in, wildtype=True, 
                                          show_track=False, modify_prefix=MODIFY_PREFIX) 
    except RuntimeError:
      # Infrequent "the entries are out of order" error for some random seqs 
        continue
    try:
        out_in = eops.extract_from_position(s_in, as_dataframe=True)
        #print (out_in.shape)
        out_in = out_in.mean()
        out_in['SEQ_ID'] = id_seq
        out_in['TARGET_NAME'] = 'ITSELF'
        columns_names = out_in.copy()
        captured_values.append( out_in  )
    except ValueError:
      # Infrequent "All arrays must be of the same length" error
        continue

df_out = pd.DataFrame([x.values.tolist() for x in captured_values], columns=out_in.index)

df_out.to_csv('PROMOTERS_SEQS.TXT', sep='\t', index=None)

Remove all head 3


['PROMOTERS'] (20013, 7)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


  0%|          | 0/3 [00:00<?, ?it/s]

## TRAINING

In [ ]:
print('Remove all head 3')
eops = EnformerOps()
eops.load_data(SEQS.extract_seq('GENERATED', 'GM12878')['SEQUENCE'].values.tolist())
load_tracks_eops(eops) #loading the tracks

ran_seqs = all_data[all_data['TAG'].apply(lambda x : x in {'training', 'test', 'validation'})].copy()
print (ran_seqs['TAG'].unique(), ran_seqs.shape)
subset_ran_seqs = ran_seqs[['chrom', 'start', 'end', 'ID']]
#ran_seq_list = subset_ran_seqs.head(3).values.tolist()
ran_seq_list = subset_ran_seqs.values.tolist()
if DEMO:
    ran_seq_list=ran_seq_list[:3]

captured_values = []
for s in tqdm(ran_seq_list): 
    #print(s)
    try:
        s_in = [s[0], int(s[1]), int(s[2])]
        id_seq = s[3] 
        list_bw = eops.generate_plot_number(0, interval_list=s_in, wildtype=True, 
                                          show_track=False, modify_prefix=MODIFY_PREFIX) 
    except RuntimeError:
      # Infrequent "the entries are out of order" error for some random seqs 
        continue
    try:
        out_in = eops.extract_from_position(s_in, as_dataframe=True)
        #print (out_in.shape)
        out_in = out_in.mean()
        out_in['SEQ_ID'] = id_seq
        out_in['TARGET_NAME'] = 'ITSELF'
        columns_names = out_in.copy()
        captured_values.append( out_in  )
    except ValueError:
      # Infrequent "All arrays must be of the same length" error
        continue

df_out = pd.DataFrame([x.values.tolist() for x in captured_values], columns=out_in.index)

df_out.to_csv('TRAINING_TEST_VALIDATION_SEQS.TXT', sep='\t', index=None)

Remove all head 3


['training' 'test' 'validation'] (47872, 7)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


  0%|          | 0/3 [00:00<?, ?it/s]

## Generated

In [ ]:
print('Remove all head 3')
eops = EnformerOps()



load_tracks_eops(eops) #loading the tracks

ran_seqs = all_data[all_data['TAG'] == 'GENERATED'].copy()
subset_ran_seqs = ran_seqs[['SEQUENCE', 'ID']]
print ( all_data[all_data['TAG'] == 'GENERATED'].shape)
print (ran_seqs['TAG'].unique())

#ran_seq_list = subset_ran_seqs.head(3).values.tolist()
ran_seq_list = subset_ran_seqs.values.tolist()
if DEMO:
    ran_seq_list=ran_seq_list[:3]

ENHANCER_REGION =  ['chrX', 48782929, 48783129]
GENE_REGION = ['chrX', 48786486, 48794411]  # 48786486,48794411
GENE_NAME = 'GATA1'

captured_values = []
captured_values_target = []
for s in tqdm(ran_seq_list): 
    
    try:
        s_in = s[1]
        id_seq = s[0]   # aways inject a sequence
        #print (id_seq)
        eops.load_data([[id_seq]]) # This will be always zero (sequence passed using insert_seq_directly)
        list_bw = eops.generate_plot_number(-1, interval_list=ENHANCER_REGION, wildtype=False, 
                                          show_track=False, modify_prefix=MODIFY_PREFIX) 
    except RuntimeError:
      # Infrequent "the entries are out of order" error for some random seqs 
        continue
    try:
        out_in = eops.extract_from_position(ENHANCER_REGION, as_dataframe=True)
        #print (out_in.shape)
        out_in = out_in.mean()
        out_in['SEQ_ID'] = id_seq
        out_in['TARGET_NAME'] = 'ENH_GATA1'
        columns_names = out_in.copy()
        captured_values.append( out_in  )
        
        out_in = eops.extract_from_position(GENE_REGION, as_dataframe=True)
        #print (out_in.shape)
        out_in = out_in.mean()
        out_in['SEQ_ID'] = id_seq
        out_in['TARGET_NAME'] = 'GATA1'
        columns_names = out_in.copy()
        captured_values_target.append( out_in  )
        
    except ValueError:
      # Infrequent "All arrays must be of the same length" error
        continue

df_out_ENH = pd.DataFrame([x.values.tolist() for x in captured_values], columns=['ENHANCER_' + x for x in   out_in.index])
df_out_GENE = pd.DataFrame([x.values.tolist() for x in captured_values_target], columns=['GENE_' + x for x in   out_in.index])

df_out = pd.concat([df_out_ENH, df_out_GENE], axis=1)

df_out.to_csv('GENERATED_SEQS.TXT', sep='\t', index=None)

In [ ]:
df_out_ENH

In [ ]:
df_out_GENE